In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import lmdb
import mindspore as ms
import mindspore.nn as nn
from mindspore.dataset import GeneratorDataset

from unimol.dataset.transforms import get_atom_vocab
from unimol.dataset.lmdb_dataset import LMDBDataset
from unimol.dataset.ligands_dataset import MoleculeDataset, molecule_collate_fn
from unimol.model import UniMol
from unimol.loss import MoleculePretrainLoss

In [4]:
ms.set_context(device_target="Ascend")

In [5]:
ligands_dataset = MoleculeDataset("./example_data/molecule/train.lmdb", "./config/atom_vocab.txt", is_train=True)

In [13]:
column_names = ["masked_atoms", "masked_coords", "masked_distance", "edge_types", 
                "target_atoms", "coordinates", "distance"]
ms_dataset = GeneratorDataset(ligands_dataset, column_names=column_names, shuffle=True,
                              python_multiprocessing=False)
#datasets = ms_dataset.split([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])
#ms_dataset = datasets[0]

In [22]:
# ms_dataset.get_dataset_size()

2354665

In [20]:
ms_dataset.num_parallel_workers

1

In [14]:
batch_dataset = ms_dataset.batch(16, per_batch_map=molecule_collate_fn)

In [ ]:
set_dynamic_columns(), set_prefetch_size(), set_enable_shared_mem()

In [22]:
import time
start = time.time()
for batch in batch_dataset.create_tuple_iterator():
    pass
total_time = time.time() - start
print(f"total traverse through dataset time: {total_time}")

total traverse through dataset time: 6.038014650344849


In [28]:
total_time / 60 / 60

3.7262354683213763

on small dataset:
parallel_workers = 8 (both), total time = 9.69 secs
best results with no parallel_workers param, total_time = 5.79 secs
on big dataset:
experiments was conducted on 1/8th part of dataset, modelling data parallel process on 8 Ascend910

In [ ]:
steps_per_epoch = ms_dataset.get_dataset_size() / 8
steps_per_epoch

In [ ]:
1000000 // steps_per_epoch + 1

In [ ]:
batch = next(dataset_iterator)

In [ ]:
masked_atoms, masked_coords, masked_distance, edge_types, target_atoms, coordinates, distance = batch

#### model debug

In [ ]:
def count_params(net):
    total_params = 0
    for param in net_trainable_params():
        total_params += np.prod(param.shape)
    return total_params

In [ ]:
atom_dict = get_atom_vocab("./config/atom_vocab.txt")
atom_vocab = atom_dict.vocab()
encoder_layers = 15
encoder_emb_dim = 512
encoder_ff_emb_dim = 2048
encoder_attention_heads = 64
gaus_kernel_channels = 128
emb_dropout = 0.9
dropout = 0.9
attention_dropout = 0.9
pooler_dropout = 0.9
max_seq_len = 256

In [ ]:
model = UniMol(
    atom_vocab, encoder_layers,
    encoder_emb_dim, encoder_ff_emb_dim,
    encoder_attention_heads, gaus_kernel_channels,
    emb_dropout, dropout, attention_dropout, 
    pooler_dropout, max_seq_len
)
model.set_train(True)

In [ ]:
token_logits, pred_coords, pred_distance, x_norm, pair_rep_norm = model(masked_atoms, masked_distance, masked_coords, edge_types, target_atoms)
tokens, coordinates, distance = (target_atoms, coordinates, distance)

In [ ]:
loss = MoleculePretrainLoss(atom_dict.vocab(), 1.0, 5.0, 10.0)

In [ ]:
losses = loss(token_logits, pred_coords, pred_distance, x_norm, pair_rep_norm, tokens, coordinates, distance)

In [ ]:
losses

In [ ]:
attention = nn.MultiHeadAttention()